<a href="https://colab.research.google.com/github/JannatulJyoti122/Thesis_Undergrade/blob/main/RQA_QA_with_ChatHistory_(1).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install langchain chromadb faiss-cpu openai tiktoken langchain_openai langchain-community wikipedia

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 4.5 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.0/19.0 MB 117.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.9/94.9 kB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 65.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.4/63.4 kB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 104.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.2/284.2 kB 27.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 94.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 438.3/438.3 kB 39.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.6/101.6 kB 10.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━

# Chunking

In [3]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

# Load the text file
file_path = "constitution.txt"  # Replace with your actual file path
with open(file_path, "r", encoding="utf-8") as file:
    text = file.read()

# Initialize the text splitter
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=400,       # max characters per chunk
    chunk_overlap=70  ,    # overlap between chunks to preserve context
    separators=["\n\n", "\n", "।", ".", " "]
)

# Split the text (pass the variable, not a string)
chunks = text_splitter.split_text(text)

# Print the chunks
for i, chunk in enumerate(chunks):
    print(f"Chunk {i+1}:\n{chunk}\n")


Chunk 1:
প্রস্তাবনা
আমরা, বাংলাদেশের জনগণ , ১৯৭১ সালের ২৬শে মার্চ স্বাধীনতা ঘোষণা করে জাতীয় মুক্তির ঐতিহাসিক সংগ্রামের মাধ্যমে স্বাধীন ও সার্বভৌম গণপ্রজাতন্ত্রী বাংলাদেশ প্রতিষ্ঠা করেছি।

Chunk 2:
আমরা অঙ্গীকার করছি, সেই সব মহান আদর্শ—যেগুলো আমাদের সাহসী জনগণকে জাতীয় মুক্তি সংগ্রামে আত্মনিয়োগ করতে এবং অনেক বীর শহীদকে জীবন উৎসর্গ করতে অনুপ্রাণিত করেছিল—যেমন: জাতীয়তাবাদ, সমাজতন্ত্র, গণতন্ত্র ও ধর্মনিরপেক্ষতা—এই আদর্শগুলোই হবে এই সংবিধানের মূলনীতি।

Chunk 3:
আমরা আরও অঙ্গীকার করছি, আমাদের রাষ্ট্রের অন্যতম লক্ষ্য হবে গণতান্ত্রিক পদ্ধতিতে একটি শোষণমুক্ত সমাজতান্ত্রিক সমাজ গঠন, যেখানে আইনের শাসন, মৌলিক মানবাধিকার এবং রাজনৈতিক, অর্থনৈতিক ও সামাজিক সমতা, স্বাধীনতা ও ন্যায্যতা সব নাগরিকের জন্য নিশ্চিত করা হবে।

Chunk 4:
আমরা জোর দিয়ে ঘোষণা করছি, যেন আমরা স্বাধীন জাতি হিসেবে উন্নতি করতে পারি এবং মানবজাতির অগ্রগতিশীল আশা ও আকাঙ্ক্ষার সঙ্গে সঙ্গতি রেখে আন্তর্জাতিক শান্তি ও সহযোগিতায় সম্পূর্ণ ভূমিকা রাখতে পারি, সে উদ্দেশ্যে এই সংবিধানের শ্রেষ্ঠত্ব বজায় রাখা এবং একে রক্ষা, সমর্থন ও নিরাপত্তা দেওয়া আ

In [4]:
from langchain_openai import OpenAIEmbeddings
from langchain.vectorstores import Chroma

# OpenAI key

In [5]:
import os
os.environ["OPENAI_API_KEY"] = ""

# Vector Embedding

In [6]:
from sentence_transformers import SentenceTransformer, LoggingHandler, models, evaluation, losses, util
from torch.utils.data import DataLoader
from sentence_transformers.datasets import ParallelSentencesDataset
import logging
import os
import numpy as np


class BanglaSentenceTransformerSmall:
    def __init__(self, model_name='intfloat/multilingual-e5-large'):
        """Initialize the model once"""
        self.model = SentenceTransformer(model_name)

    def encode(self, sentences):
        """Encode a list of sentences"""
        sentence_embeddings = self.model.encode(sentences)
        return dict(zip(sentences, sentence_embeddings))

    def similarity(self, embedding1, embedding2):
        """Compute cosine similarity"""
        return util.pytorch_cos_sim(embedding1, embedding2)

    def train(self, path, output_path="output/bangla-sentence-transformer", load_pretrained=True):
        """Train a new student model using knowledge distillation from a teacher"""
        logging.basicConfig(format='%(asctime)s - %(message)s',
                            datefmt='%Y-%m-%d %H:%M:%S',
                            level=logging.INFO,
                            handlers=[LoggingHandler()])

        teacher_model = SentenceTransformer('intfloat/multilingual-e5-large')
        student_model_name = 'xlm-roberta-base'
        max_seq_length = 128
        batch_size = 64
        max_sentences = 500000
        max_sentence_length = 300
        num_epochs = 5
        warmup_steps = 10000
        eval_steps = 500

        # Student model creation
        if load_pretrained:
            student_model = SentenceTransformer('intfloat/multilingual-e5-large')
        else:
            word_embedding_model = models.Transformer(student_model_name, max_seq_length=max_seq_length)
            pooling_model = models.Pooling(word_embedding_model.get_word_embedding_dimension())
            student_model = SentenceTransformer(modules=[word_embedding_model, pooling_model])

        # Dataset loading
        train_data = ParallelSentencesDataset(student_model=student_model, teacher_model=teacher_model,
                                              batch_size=batch_size, use_embedding_cache=False)
        train_data.load_data(path, max_sentences=max_sentences,
                             max_sentence_length=max_sentence_length)
        train_dataloader = DataLoader(train_data, shuffle=True, batch_size=batch_size)
        train_loss = losses.MSELoss(model=student_model)

        # Evaluation setup
        src_sentences, trg_sentences = [], []
        with open(path, 'r', encoding='utf-8') as f:
            for line in f:
                parts = line.strip().split('\t')
                if len(parts) == 2 and all(parts):
                    src_sentences.append(parts[0])
                    trg_sentences.append(parts[1])
                if len(src_sentences) >= eval_steps:
                    break

        evaluator = evaluation.MSEEvaluator(src_sentences, trg_sentences, name=os.path.basename(path),
                                            teacher_model=teacher_model, batch_size=batch_size)

        # Training
        student_model.fit(
            train_objectives=[(train_dataloader, train_loss)],
            evaluator=evaluation.SequentialEvaluator([evaluator], main_score_function=np.mean),
            epochs=num_epochs,
            warmup_steps=warmup_steps,
            evaluation_steps=eval_steps,
            output_path=output_path,
            save_best_model=True,
            optimizer_params={'lr': 2e-5, 'eps': 1e-6, 'correct_bias': False}
        )


# Vector Store

In [7]:
from langchain.docstore.document import Document
import chromadb
from langchain.embeddings import SentenceTransformerEmbeddings
 # Assuming this is your custom class

# 1. Initialize the Bangla Sentence Transformer
model = BanglaSentenceTransformerSmall()

# 2. Encode the chunks
embeddings_dict = model.encode(chunks)

# 3. Prepare documents with the chunk text
documents = [Document(page_content=chunk) for chunk in embeddings_dict.keys()]

# 4. Set up ChromaDB client
client = chromadb.Client()

# 5. Try to get the collection if it already exists
collection_name = "bangla_embeddings"
try:
    collection = client.get_collection(collection_name)
except Exception as e:
    # If collection doesn't exist, create it
    collection = client.create_collection(collection_name)

# 6. Add the sentence embeddings to ChromaDB collection
for idx, (sentence, embedding) in enumerate(embeddings_dict.items()):
    # Ensure idx is defined within the loop, no error in 'for' loop
    collection.add(
        ids=[f"doc_{idx}"],             # Unique ID for each document
        documents=[sentence],           # The sentence content
        metadatas=[{"source": "constitution"}],  # Added a key-value pair for metadata
        embeddings=[embedding]          # The corresponding embedding
    )
#client.persist()

print(f"Stored {len(chunks)} chunks in Chroma vector store.")

#print(f"Stored {len(chunks)} chunks in Chroma vector store.")


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/387 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/160k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/57.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/690 [00:00<?, ?B/s]

...

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/280 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/201 [00:00<?, ?B/s]

Stored 324 chunks in Chroma vector store.


In [8]:
from chromadb.config import Settings
client = chromadb.PersistentClient(path="my_chroma_db")


# Test Retrieval

In [9]:
query = "কখন রাষ্ট্রদ্রোহিতা হবে?"
query_embedding = model.encode([query])[query]  # Get the embedding

results = collection.query(
    query_embeddings=[query_embedding],
    n_results=7,
    include=["documents", "distances"]  # Important: request distances
)

# Display each result with its similarity score
for doc, score in zip(results['documents'][0], results['distances'][0]):
    print(f"Document: {doc}")
    print(f"Similarity Score: {score:.4f}\n")

Document: ৭ক। সংবিধান বাতিল, স্থগিতকরণ, রাষ্ট্রদ্রোহিতা ইত্যাদি অপরাধ :
১। কেউ যদি শক্তি ব্যবহার করে, বা অন্য কোনো বেআইনি উপায়ে—
এই সংবিধান বা এর কোনো অংশ বাতিল বা স্থগিত করে, বা সেটা করার চেষ্টা করে বা ষড়যন্ত্র করে, বা জনগণের মধ্যে সংবিধানের প্রতি বিশ্বাস কমানোর চেষ্টা করে—
তাহলে সেটি রাষ্ট্রদ্রোহিতা হিসেবে গণ্য হবে। সেই ব্যক্তি রাষ্ট্রদ্রোহের অপরাধে অপরাধী বলে ধরা হবে।
Similarity Score: 0.3422

Document: দ্বিতীয় ভাগ, রাষ্ট্র পরিচালনার মূলনীতি
(৮ )(১) জাতীয়তাবাদ, সমাজতন্ত্র, গণতন্ত্র ও ধর্মনিরপেক্ষতা - এই নীতিসমূহ এবং এদের থেকে উদ্ভূত অন্যান্য নীতিসমূহ রাষ্ট্র পরিচালনার মূলনীতি হিসেবে গণ্য হবে।
Similarity Score: 0.3931

Document: ধর্ম, প্রভৃতি কারণে বৈষম্য
২৮।(১) কোনো নাগরিকের প্রতি কেবল ধর্ম, গোষ্ঠী, বর্ণ, লিঙ্গ বা জন্মস্থানের কারণে রাষ্ট্র বৈষম্য করবে না।
(২) রাষ্ট্র ও সমাজের সব স্তরে নারী ও পুরুষের সমান অধিকার নিশ্চিত করবে।
(৩) ধর্ম, গোষ্ঠী, বর্ণ, লিঙ্গ বা জন্মস্থানের কারণে কোনো নাগরিককে বিনোদন বা বিশ্রামের স্থানে প্রবেশ বা শিক্ষা প্রতিষ্ঠানে ভর্তিতে কোনো বাধা, শর্ত বা অযোগ্যতা দেও

In [11]:

query = "সংবিধানের মূলনীতি?"
query_embedding = model.encode([query])[query]  # Get the embedding

results = collection.query(
    query_embeddings=[query_embedding],
    n_results=7,
    include=["documents", "distances"]  # Important: request distances
)

# Display each result with its similarity score
for doc, score in zip(results['documents'][0], results['distances'][0]):
    print(f"Document: {doc}")
    print(f"Similarity Score: {score:.4f}\n")

Document: আমরা অঙ্গীকার করছি, সেই সব মহান আদর্শ—যেগুলো আমাদের সাহসী জনগণকে জাতীয় মুক্তি সংগ্রামে আত্মনিয়োগ করতে এবং অনেক বীর শহীদকে জীবন উৎসর্গ করতে অনুপ্রাণিত করেছিল—যেমন: জাতীয়তাবাদ, সমাজতন্ত্র, গণতন্ত্র ও ধর্মনিরপেক্ষতা—এই আদর্শগুলোই হবে এই সংবিধানের মূলনীতি।
Similarity Score: 0.3194

Document: দ্বিতীয় ভাগ, রাষ্ট্র পরিচালনার মূলনীতি
(৮ )(১) জাতীয়তাবাদ, সমাজতন্ত্র, গণতন্ত্র ও ধর্মনিরপেক্ষতা - এই নীতিসমূহ এবং এদের থেকে উদ্ভূত অন্যান্য নীতিসমূহ রাষ্ট্র পরিচালনার মূলনীতি হিসেবে গণ্য হবে।
Similarity Score: 0.3262

Document: তৃতীয় ভাগ, মৌলিক অধিকার
Similarity Score: 0.3424

Document: ৭খ। সংবিধানের মৌলিক বিধান অপরিবর্তনীয় (সহজ ভাষায়):
যদিও সংবিধানের ১৪২ নম্বর অনুচ্ছেদে সংবিধান সংশোধনের নিয়ম আছে,
তবুও—
সংবিধানের প্রস্তাবনা (preamble),প্রথম, দ্বিতীয়, তৃতীয় এবং একাদশ ভাগ,নবম-ক ভাগের নির্দিষ্ট অনুচ্ছেদএবং সংবিধানের মৌলিক কাঠামোর সঙ্গে সম্পর্কিত অনুচ্ছেদগুলোকোনভাবেই পরিবর্তন, সংশোধন, বাদ দেওয়া বা প্রতিস্থাপন করা যাবেনা।
Similarity Score: 0.3497

Document: ৭। সংবিধানের প্রাধান্য
(১) দেশের সব ক্ষ

In [12]:
from langchain_core.runnables import RunnableParallel, RunnablePassthrough, RunnableLambda
from langchain_core.output_parsers import StrOutputParser

In [13]:
from langchain_core.runnables import RunnableLambda, RunnablePassthrough, RunnableParallel
from langchain_core.prompts import PromptTemplate
from langchain.memory import ChatMessageHistory
from langchain_core.messages import HumanMessage, AIMessage
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser
from sentence_transformers import SentenceTransformer

In [14]:
# Step 1: Initialize models
#embedding_model = SentenceTransformer('sagorsarker/bangla-bert-base')  # Better for Bengali
llm = ChatOpenAI(
    openai_api_key="",
    model_name="gpt-3.5-turbo-0125",  # Use a model with better multilingual support
    temperature=0.2
)
parser = StrOutputParser()

# Step 2: Initialize Chat History
chat_history = ChatMessageHistory()

# Format ChatHistory

In [15]:
def format_chat_history(history, max_messages=10):
    """Convert chat history into a formatted string."""
    messages = history.messages[-max_messages:]
    formatted = ""
    for msg in messages:
        if isinstance(msg, HumanMessage):
            formatted += f"User: {msg.content}\n"
        elif isinstance(msg, AIMessage):
            formatted += f"Assistant: {msg.content}\n"
    return formatted

# Format Documents

In [16]:
# Step 4: Define a function to format documents
def format_docs(results):
    """Format vector store results into a string."""
    #print("Raw results structure:", results)  # Debug full results
    documents = results.get('documents', [[]])[0]  # Get first list of documents
    #print("Extracted documents:", documents)  # Debug extracted documents
    if not documents:
        print("Warning: No documents retrieved!")
        return ""
    formatted = "\n".join([str(doc) for doc in documents if doc])  # Ensure no empty strings
    #print("Formatted context:", formatted[:200], "...")  # Debug formatted output
    if not formatted.strip():
        print("Warning: Formatted context is empty after processing!")
    return formatted  # Limit context length

# Query Embedding

In [30]:
# Step 5: Define the vector store query function
def query_vector_store(question):
    embedding = model.encode([question])[question]
    results = collection.query(query_embeddings=[embedding], n_results=7)
    #print("Retrieved documents in query_vector_store:", results.get('documents', [[]])[0])
    #print("Retrieved documents in query_vector_store:  ================================================================================= \n")
    return results

In [31]:
# Step 6: Wrap components in LangChain Runnables
query_chain = RunnableLambda(query_vector_store)
context_chain = query_chain | RunnableLambda(format_docs)

# Prompt

In [32]:
# Step 7: Update the Prompt Template
prompt = PromptTemplate(
    template="""
      আপনি বাংলাদেশের সংবিধানের একটি সহায়ক সহকারী।
      শুধুমাত্র নিচের প্রাসঙ্গিক অংশের ভিত্তিতে প্রশ্নের সঠিক এবং সংক্ষিপ্ত উত্তর দিন।
      চ্যাট ইতিহাস:
      {history}
      প্রাসঙ্গিক অংশ:
      {context}
      প্রশ্ন:
      {question}
      উত্তর শুধুমাত্র প্রাসঙ্গিক অংশ থেকে নিন। কোনো বাহ্যিক তথ্য, অনুমান, বা সাধারণ জ্ঞান ব্যবহার করবেন না।
      যদি উত্তর প্রাসঙ্গিক অংশে না থাকে, তবে বলুন: "উত্তর প্রাসঙ্গিক অংশে পাওয়া যায়নি।"
      উত্তর বাংলায়, সংক্ষিপ্ত এবং সঠিক হতে হবে।
    """,
    input_variables=['history', 'context', 'question']
)

# Parallel Chain

In [33]:
# Step 8: Update the parallel chain to include history
def get_history(_):
    return format_chat_history(chat_history)

parallel_chain = RunnableParallel({
    'history': RunnableLambda(get_history),
    'context': context_chain,
    'question': RunnablePassthrough()
})

# Main Chain Formation

In [34]:

# Step 9: Define the main chain with debugging
def debug_invoke(input):
    #print("Prompt input:", input)
    return prompt.invoke(input)

main_chain = parallel_chain | debug_invoke | llm | parser

# Funtion to ask question which invoke the main_chain

In [35]:
# Step 10: Function to invoke the chain and update history
def ask_question(question):
    try:
        chat_history.add_message(HumanMessage(content=question))
        result = main_chain.invoke(question)
        #print("Raw LLM output:", result)
        chat_history.add_message(AIMessage(content=result))
        return result
    except Exception as e:
        print(f"Error during generation: {e}")
        return None

# Printing Result

In [46]:
# Example usage
question = "কখন স্বাধীনতার ঘোষণা দেওয়া হয়?"
response = ask_question(question)
print("Response:", response)

# Follow-up question
follow_up = "কেন ঘোষণা দেওয়া হয়?"
response = ask_question(follow_up)
print("Follow-up Response:", response)

Response: স্বাধীনতার ঘোষণা ১৯৭১ সালের ২৬শে মার্চে দেওয়া হয়।
Follow-up Response: উত্তর প্রাসঙ্গিক অংশে দেওয়া হয়েছে যে, জরুরী অবস্থা ঘোষণা করা হয় বাংলাদেশের সংবিধানের একটি জরুরী বিধানাবলী অনুযায়ী, যদি যুদ্ধ, বহিরাগত আক্রমণ বা অভ্যন্তরীণ বিশৃঙ্খলার কারণে বাংলাদেশ বা এর কোনো অংশের নিরাপত্তা বা অর্থনৈতিক জীবন বিপন্ন হয়ে পড়ে।


In [37]:

# Example usage
question = "সংবিধানে মানবাধিকার নিয়ে কী বলা আছে?"
response = ask_question(question)
print("Response:", response)

Response: মানবাধিকার সংবিধানে অত্যন্ত গুরুত্বপূর্ণ বিষয় হিসেবে বিবেচিত হয়েছে। এটি বলে, কোনো ব্যক্তিকে আইন অনুযায়ী ছাড়া তার জীবন বা ব্যক্তিস্বাধীনতা থেকে বঞ্চিত করা যাবে না। এটি মানুষের মর্যাদা ও মূল্যকে সম্মান করার জন্য উত্তরণ করে।


In [38]:


# Example usage
question = "কখন রাষ্ট্রদ্রোহিতা হবে?"
response = ask_question(question)
print("Response:", response)

Response: এটি রাষ্ট্রদ্রোহিতা হবে যখন কেউ শক্তি ব্যবহার করে, বা অন্য কোনো বেআইনি উপায়ে সংবিধান বা এর কোনো অংশ বাতিল বা স্থগিত করে, বা সেটা করার চেষ্টা করে বা ষড়যন্ত্র করে, বা জনগণের মধ্যে সংবিধানের প্রতি বিশ্বাস কমানোর চেষ্টা করে।


In [39]:

# Example usage
question = "বাংলাদেশের জাতীয় প্রতীক "
response = ask_question(question)
print("Response:", response)

Response: জাতীয় প্রতীকের বর্ণনা: পানির উপর ভাসছে জাতীয় ফুল শাপলা। চারপাশে ঘিরে আছে ধানের শীষ। উপরে আছে তিনটি পাটপাতা এবং দুই পাশে দুটি তারা (তারকা)।


In [40]:

# Example usage
question = "এটির চারপাশে কী রয়েছে?  "
response = ask_question(question)
print("Response:", response)

Response: জাতীয় প্রতীকের চারপাশে ধানের শীষ এবং তিনটি পাটপাতা এবং দুই পাশে দুটি তারা (তারকা) রয়েছে।


In [41]:


# Example usage
question = "এটির  চারপাশে কয়টা তারকা রয়েছে? "
response = ask_question(question)
print("Response:", response)

Response: দুটি তারকা।


In [42]:

# Example usage
question = "সংবিধানে মূলনীতি?  "
response = ask_question(question)
print("Response:", response)

Response: সংবিধানের মূলনীতি হলো জাতীয়তাবাদ, সমাজতন্ত্র, গণতন্ত্র ও ধর্মনিরপেক্ষতা। এই নীতিসমূহ এবং এদের থেকে উদ্ভূত অন্যান্য নীতিসমূহ রাষ্ট্র পরিচালনার মূলনীতি হিসেবে গণ্য হবে।


In [47]:

# Example usage
question = "রাষ্ট্রের  মূলনীতি?  "
response = ask_question(question)
print("Response:", response)

Response: উত্তর: রাষ্ট্রের মূলনীতি হল জাতীয়তাবাদ, সমাজতন্ত্র, গণতন্ত্র ও ধর্মনিরপেক্ষতা। (প্রাসঙ্গিক অংশ থেকে নেওয়া হয়েছে)।


# Testing with a dataset

In [48]:
import pandas as pd

# Data
data = {
    "Question": [
        "বাংলাদেশের সংবিধানের প্রস্তাবনাতে কি উল্লেখ করা হয়েছে?",
        "দেশের সর্বোচ্চ আইন কী?",
        "কি কারণে সাংবিধানিক পদক্ষেপ নেওয়া যেতে পারে?",
        "বাংলাদেশের জাতীয় সঙ্গীত কী?",
        "সংবিধানে রাষ্ট্রের মূলনীতি কী বলা হয়েছে?",
        "সংবিধানের সংশোধন কিভাবে হয়?"
    ],
    "Context": [
        "বাংলাদেশের সংবিধান দেশের সর্বোচ্চ আইন।",
        "বাংলাদেশের সংবিধান দেশের সর্বোচ্চ আইন।",
        "যদি সংবিধান লঙ্ঘন হয়, তবে সাংবিধানিক পদক্ষেপ নেওয়া যেতে পারে।",
        "বাংলাদেশের জাতীয় সঙ্গীত 'আমার সোনার বাংলা'।",
        "রাষ্ট্রের মূলনীতি হিসেবে গণতন্ত্র, সমাজবাদী রাষ্ট্রব্যবস্থা, এবং ধর্মনিরপেক্ষতা বলা হয়েছে।",
        "সংবিধানের সংশোধন করতে সংসদে বিশেষ প্রস্তাব পাস করতে হয়।"
    ],
    "Answer": [
        "বাংলাদেশের সংবিধানের প্রস্তাবনাতে বাংলাদেশের স্বাধীনতা, গণতান্ত্রিক মূল্যবোধ, এবং সমাজবাদী লক্ষ্যগুলি সম্পর্কে উল্লেখ করা হয়েছে।",
        "বাংলাদেশের সংবিধান দেশের সর্বোচ্চ আইন।",
        "সংবিধান লঙ্ঘন হলে সাংবিধানিক পদক্ষেপ নেওয়া যেতে পারে।",
        "'আমার সোনার বাংলা' জাতীয় সঙ্গীত।",
        "গণতন্ত্র, সমাজবাদী রাষ্ট্রব্যবস্থা, ধর্মনিরপেক্ষতা।",
        "সংসদে বিশেষ প্রস্তাব পাসের মাধ্যমে সংশোধন হয়।"
    ]
}

# Create a DataFrame
df = pd.DataFrame(data)

# Save to CSV file
df.to_csv('bangla_constitution_part1_question_ans.csv', index=False)

print("CSV file with questions, context, and answers has been created successfully.")


CSV file with questions, context, and answers has been created successfully.


In [49]:
# prompt: load the csv file

import pandas as pd

# Assuming the CSV file 'bangla_constitution_qa.csv' is in the current directory
try:
    df = pd.read_csv('bangla_constitution_part1_question_ans.csv.csv')
    print("CSV file loaded successfully.")
    # Now you can work with the DataFrame 'df'
    print(df.head())  # Print the first few rows to verify

except FileNotFoundError:
    print("Error: 'bangla_constitution_part1_question_ans.csv.csv' not found in the current directory.")
except pd.errors.EmptyDataError:
    print("Error: 'bangla_constitution_part1_question_ans.csv' is empty.")
except pd.errors.ParserError:
    print("Error: Unable to parse the CSV file. Check its format.")
except Exception as e:
    print(f"An unexpected error occurred: {e}")


Error: 'bangla_constitution_part1_question_ans.csv.csv' not found in the current directory.


In [50]:
from sklearn.metrics.pairwise import cosine_similarity
#modell = SentenceTransformer('shihab17/bangla-sentence-transformer')
def evaluate_retrieval_accuracy(dataset, model, k=5, similarity_threshold=0.7):
    retrieval_hits = 0
    total_questions = len(dataset)

    for _, example in dataset.iterrows():  # Iterate over rows of the DataFrame
        question = example['Question']
        context = example['Context']
        true_answer = example['Answer']

        # Generate embedding for the true answer
        #true_answer_embedding = modell.encode(true_answer)
        #new_question = normalize_question(question)
        # Normalize the question
        #new_question = normalize_question(question)
        #query_embedding = model.encode([question])[question] # Get the query embedding

        # Retrieve top-k documents
        generated_ans=main_chain.invoke(question);

        print(question)
        print('question: ', question)
        print("answer ",true_answer)
        print("gen_ans ",generated_ans)

        # Calculate the similarity between the concatenated content and the true answer
        #generated_ans_embedding = modell.encode(generated_ans)
        #similarity = cosine_similarity([true_answer_embedding], [generated_ans_embedding])[0][0]

        #print(similarity)
        print("--------------------------------------------------------------------------------------\n\n")
        # If similarity exceeds threshold, consider it a hit
        #if similarity > similarity_threshold:
           # retrieval_hits += 1

    #retrieval_accuracy = retrieval_hits / total_questions
    #return retrieval_accuracy

# Example usage with your DataFrame (df):
retrieval_accuracy = evaluate_retrieval_accuracy(df, model)
#print(f"Retrieval Accuracy: {retrieval_accuracy:.2f}")


বাংলাদেশের সংবিধানের প্রস্তাবনাতে কি উল্লেখ করা হয়েছে?
question:  বাংলাদেশের সংবিধানের প্রস্তাবনাতে কি উল্লেখ করা হয়েছে?
answer  বাংলাদেশের সংবিধানের প্রস্তাবনাতে বাংলাদেশের স্বাধীনতা, গণতান্ত্রিক মূল্যবোধ, এবং সমাজবাদী লক্ষ্যগুলি সম্পর্কে উল্লেখ করা হয়েছে।
gen_ans  উত্তর: সংবিধানের প্রস্তাবনাতে উল্লেখ করা হয়েছে যে, আমরা, বাংলাদেশের জনগণ, ১৯৭১ সালের ২৬শে মার্চ স্বাধীনতা ঘোষণা করে জাতীয় মুক্তির ঐতিহাসিক সংগ্রামের মাধ্যমে স্বাধীন ও সার্বভৌম গণপ্রজাতন্ত্রী বাংলাদেশ প্রতিষ্ঠা করেছি।
--------------------------------------------------------------------------------------


দেশের সর্বোচ্চ আইন কী?
question:  দেশের সর্বোচ্চ আইন কী?
answer  বাংলাদেশের সংবিধান দেশের সর্বোচ্চ আইন।
gen_ans  সংবিধান।
--------------------------------------------------------------------------------------


কি কারণে সাংবিধানিক পদক্ষেপ নেওয়া যেতে পারে?
question:  কি কারণে সাংবিধানিক পদক্ষেপ নেওয়া যেতে পারে?
answer  সংবিধান লঙ্ঘন হলে সাংবিধানিক পদক্ষেপ নেওয়া যেতে পারে।
gen_ans  সাংবিধানিক পদক্ষেপ নেওয়া হতে পারে যদ

# Evaluation

In [51]:
!pip install ragas

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.9/190.9 kB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 4.3 MB/s eta 0:00:00


In [52]:
import pandas as pd
from datasets import Dataset
from ragas import evaluate
from ragas.metrics import faithfulness, answer_correctness

# Load your CSV dataset
df = pd.read_csv('bangla_constitution_part1_question_ans.csv')

questions = df['Question'].tolist()
ground_truth_answers = df['Answer'].tolist()
retrieved_contexts = df['Context'].apply(lambda x: [x]).tolist()

# Generate answers by calling your LLM for each question
generated_answers = []
for q in questions:
    # Replace with your actual LLM call, e.g. main_chain.invoke(q)
    answer = main_chain.invoke(q)  # Make sure main_chain.invoke returns a string answer
    #print(q)
    #print(answer)
    #print("\n\n")
    generated_answers.append(answer)

# Prepare dataset for evaluation
data_samples = {
    'question': questions,
    'answer': generated_answers,
    'contexts': retrieved_contexts,
    'ground_truth': ground_truth_answers
}
dataset = Dataset.from_dict(data_samples)

# Evaluate with RAGAS metrics
score = evaluate(dataset, metrics=[faithfulness, answer_correctness])

# Convert scores to pandas DataFrame for per-question scores
df_scores = score.to_pandas()

# Print scores question-wise
for i, question in enumerate(questions):
    correctness = df_scores.loc[i, 'answer_correctness']
    faithful = df_scores.loc[i, 'faithfulness']
    print(f"Question {i+1}: {question}")
    print(f"Answer {i+1}: {ground_truth_answers[i]}")
    print(f"Generated Answer: {generated_answers[i]}")
    print(f"Answer Correctness Score: {correctness:.3f}")
    print(f"Faithfulness Score: {faithful:.3f}")
    print('-' * 50)


Evaluating:   0%|          | 0/12 [00:00<?, ?it/s]

Question 1: বাংলাদেশের সংবিধানের প্রস্তাবনাতে কি উল্লেখ করা হয়েছে?
Answer 1: বাংলাদেশের সংবিধানের প্রস্তাবনাতে বাংলাদেশের স্বাধীনতা, গণতান্ত্রিক মূল্যবোধ, এবং সমাজবাদী লক্ষ্যগুলি সম্পর্কে উল্লেখ করা হয়েছে।
Generated Answer: উত্তর: বাংলাদেশের সংবিধানের প্রস্তাবনাতে উল্লেখ করা হয়েছে যে, আমরা বাংলাদেশের জনগণ, ১৯৭১ সালের ২৬শে মার্চ স্বাধীনতা ঘোষণা করে জাতীয় মুক্তির ঐতিহাসিক সংগ্রামের মাধ্যমে স্বাধীন ও সার্বভৌম গণপ্রজাতন্ত্রী বাংলাদেশ প্রতিষ্ঠা করেছি।
Answer Correctness Score: 0.661
Faithfulness Score: 0.000
--------------------------------------------------
Question 2: দেশের সর্বোচ্চ আইন কী?
Answer 2: বাংলাদেশের সংবিধান দেশের সর্বোচ্চ আইন।
Generated Answer: সংবিধান।
Answer Correctness Score: 0.714
Faithfulness Score: 1.000
--------------------------------------------------
Question 3: কি কারণে সাংবিধানিক পদক্ষেপ নেওয়া যেতে পারে?
Answer 3: সংবিধান লঙ্ঘন হলে সাংবিধানিক পদক্ষেপ নেওয়া যেতে পারে।
Generated Answer: সাংবিধানিক পদক্ষেপ নেওয়া হতে পারে যদি জরুরী অবস্থা ঘোষণা করা হয় এবং সংবিধ